In [5]:
import os
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from torch.optim.lr_scheduler import MultiStepLR

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [4]:
train_dataset = datasets.ImageNet(root='../data/imagenet-mini', split='train', transform=train_transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = datasets.ImageNet(root='../data/imagenet-mini', split='val', transform=val_transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/imagenet-mini/train/'

In [ ]:
model_path = '../models/model_ResNet_scripted.pt'
model = torch.jit.load(model_path, map_location=device).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30, 60, 90], gamma=0.1)

In [ ]:
num_epochs = 100

In [ ]:
log_df = pd.DataFrame(columns=['epoch', 'accuracy', 'val_loss'])

In [ ]:
for epoch in tqdm(range(num_epochs), desc="Total Training Progress"):
    model.train()
    
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    scheduler.step()

    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
    accuracy = 100 * correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Accuracy: {accuracy:.2f}%, Validation Loss: {val_loss/len(val_loader):.4f}')

    log_df = log_df.append({'epoch': epoch+1, 'accuracy': accuracy, 'val_loss': val_loss/len(val_loader)}, ignore_index=True)

log_df.to_csv('training_log.csv', index=False)


scripted_trained_model = torch.jit.script(model)
scripted_trained_model.save("trained_models/trained_model_scripted_ResNet.pt")